<a href="https://colab.research.google.com/github/manobhisriram1/Bank-Management-System/blob/master/Skin_Cancer_Classification_using_CNN_and_Transfer_Learning_(ResNet50).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import timm  # Vision Transformer Library


In [ ]:
!pip install kaggle


In [ ]:
from google.colab import files
files.upload()  # Select kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dimmitimanobhisriram","key":"393cba229fff6cbc66c1cd3ee29352ad"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000


Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.20G/5.20G [00:42<00:00, 244MB/s]
100% 5.20G/5.20G [00:42<00:00, 132MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("skin-cancer-mnist-ham10000.zip", "r") as zip_ref:
    zip_ref.extractall("/content/HAM10000")


In [ ]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split


In [ ]:
# Define base dataset path
base_dir = "/content/HAM10000"
img_dir = base_dir  # Images are directly in HAM10000 folder
csv_path = os.path.join(base_dir, "HAM10000_metadata.csv")

# Read metadata CSV file
metadata = pd.read_csv(csv_path)

# Get unique labels (skin cancer types)
labels = metadata['dx'].unique()

# Create train & test directories
train_dir = "/content/train"
test_dir = "/content/test"

for label in labels:
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)


In [ ]:
# Split data (80% train, 20% test) while maintaining class distribution
train_files, test_files = train_test_split(
    metadata, test_size=0.2, stratify=metadata['dx'], random_state=42
)


In [ ]:
# Move training images to respective folders
for _, row in train_files.iterrows():
    src_path = os.path.join(img_dir, row['image_id'] + ".jpg")
    dst_path = os.path.join(train_dir, row['dx'], row['image_id'] + ".jpg")
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

# Move testing images to respective folders
for _, row in test_files.iterrows():
    src_path = os.path.join(img_dir, row['image_id'] + ".jpg")
    dst_path = os.path.join(test_dir, row['dx'], row['image_id'] + ".jpg")
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

print("✅ Dataset organized into Train and Test folders!")


✅ Dataset organized into Train and Test folders!


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader


In [ ]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match input size
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize for stable training
])


In [ ]:
import os

# Check if train and test folders exist
print("Train folder exists:", os.path.exists(train_dir))
print("Test folder exists:", os.path.exists(test_dir))

# List files inside train directory (should contain subfolders)
print("Train directory contents:", os.listdir(train_dir) if os.path.exists(train_dir) else "Not found")

# List files inside test directory
print("Test directory contents:", os.listdir(test_dir) if os.path.exists(test_dir) else "Not found")


Train folder exists: True
Test folder exists: True
Train directory contents: ['bcc', 'mel', 'bkl', 'akiec', 'vasc', 'nv', 'df']
Test directory contents: ['bcc', 'mel', 'bkl', 'akiec', 'vasc', 'nv', 'df']


In [ ]:
# Check if there are images inside one of the label folders
label = "mel"  # Change this to any other class like 'nv', 'bcc', etc.
label_path = os.path.join(train_dir, label)

if os.path.exists(label_path):
    print(f"Files in {label_path}: {os.listdir(label_path)[:5]}")  # Show first 5 images
else:
    print(f"Folder {label_path} does not exist!")


Files in /content/train/mel: []


In [ ]:
import os

base_dir = "/content/HAM10000"  # Change this if you used a different path

# List first 10 image files in the original dataset
print("Sample images from the original dataset:", os.listdir(base_dir)[:10])


Sample images from the original dataset: ['HAM10000_images_part_2', 'HAM10000_metadata.csv', 'hmnist_28_28_L.csv', 'hmnist_8_8_L.csv', 'hmnist_28_28_RGB.csv', 'hmnist_8_8_RGB.csv', 'ham10000_images_part_1', 'HAM10000_images_part_1', 'ham10000_images_part_2']


In [ ]:
import os

base_dir = "/content/HAM10000"

# Check inside part_1 folder
part1_path = os.path.join(base_dir, "HAM10000_images_part_1")
part2_path = os.path.join(base_dir, "HAM10000_images_part_2")

print("Images in Part 1:", os.listdir(part1_path)[:5] if os.path.exists(part1_path) else "Not found")
print("Images in Part 2:", os.listdir(part2_path)[:5] if os.path.exists(part2_path) else "Not found")


Images in Part 1: ['ISIC_0028910.jpg', 'ISIC_0026432.jpg', 'ISIC_0028549.jpg', 'ISIC_0024929.jpg', 'ISIC_0025829.jpg']
Images in Part 2: ['ISIC_0033103.jpg', 'ISIC_0030648.jpg', 'ISIC_0032506.jpg', 'ISIC_0030233.jpg', 'ISIC_0031943.jpg']


In [ ]:
import shutil

# Create a folder to store all images together
all_images_dir = os.path.join(base_dir, "all_images")
os.makedirs(all_images_dir, exist_ok=True)

# Move images from part_1
for img in os.listdir(part1_path):
    shutil.move(os.path.join(part1_path, img), os.path.join(all_images_dir, img))

# Move images from part_2
for img in os.listdir(part2_path):
    shutil.move(os.path.join(part2_path, img), os.path.join(all_images_dir, img))

print("✅ All images moved to:", all_images_dir)


✅ All images moved to: /content/HAM10000/all_images


In [ ]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

# Define dataset paths
base_dir = "/content/HAM10000"
all_images_dir = os.path.join(base_dir, "all_images")  # All images are here now
csv_path = os.path.join(base_dir, "HAM10000_metadata.csv")  # Metadata file

# Read metadata CSV file
metadata = pd.read_csv(csv_path)

# Define train/test directories
train_dir = "/content/train"
test_dir = "/content/test"

# Create class folders
labels = metadata['dx'].unique()
for label in labels:
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)

# Split data (80% Train, 20% Test)
train_files, test_files = train_test_split(metadata, test_size=0.2, stratify=metadata['dx'], random_state=42)

# Move images into train and test folders
for _, row in train_files.iterrows():
    src_path = os.path.join(all_images_dir, row['image_id'] + ".jpg")
    dst_path = os.path.join(train_dir, row['dx'], row['image_id'] + ".jpg")
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

for _, row in test_files.iterrows():
    src_path = os.path.join(all_images_dir, row['image_id'] + ".jpg")
    dst_path = os.path.join(test_dir, row['dx'], row['image_id'] + ".jpg")
    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

print("✅ Dataset successfully organized into Train and Test folders!")


✅ Dataset successfully organized into Train and Test folders!


In [ ]:
import os

train_dir = "/content/train"
test_dir = "/content/test"

# Count images in train and test folders
num_train = sum([len(files) for _, _, files in os.walk(train_dir)])
num_test = sum([len(files) for _, _, files in os.walk(test_dir)])

print(f"✅ Train images: {num_train}")
print(f"✅ Test images: {num_test}")


✅ Train images: 8012
✅ Test images: 2003


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Load train and test datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check dataset size
print(f"✅ Train Data: {len(train_dataset)} images")
print(f"✅ Test Data: {len(test_dataset)} images")


✅ Train Data: 8012 images
✅ Test Data: 2003 images


In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define CNN Model
class SkinCancerCNN(nn.Module):
    def __init__(self):
        super(SkinCancerCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 7)  # 7 classes in HAM10000 dataset
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Initialize model
model = SkinCancerCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SkinCancerCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=7, bias=True)
  )
)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ Training complete!")


Epoch [1/10], Loss: 0.9614
Epoch [2/10], Loss: 0.8347


In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
test_accuracy = accuracy_score(all_labels, all_preds)
print(f"✅ Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
import torchvision.models as models

# Load ResNet-50 with pretrained weights
model = models.resnet50(pretrained=True)

# Modify the final layer to match our 7-class classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7 classes in HAM10000 dataset

# Move to GPU if available
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ Transfer Learning Training Complete!")


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds)
print(f"✅ New Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
# Load EfficientNet-B0 (pretrained)
model = models.efficientnet_b0(pretrained=True)

# Modify the final layer
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 7)  # 7 classes

# Move to GPU
model = model.to(device)

# Define loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import accuracy_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load EfficientNet-B0 (pretrained)
model = models.efficientnet_b0(pretrained=True)

# Modify the final layer to match our 7-class classification
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 7)  # 7 classes in HAM10000 dataset

# Move to GPU
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
num_epochs = 10  # Increase if needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ EfficientNet Training Complete!")


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds)
print(f"✅ EfficientNet Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
# Load ResNet-50
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Linear(resnet.fc.in_features, 7)  # Modify final layer

# Load EfficientNet-B0
efficientnet = models.efficientnet_b0(pretrained=True)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 7)

# Move models to GPU
resnet = resnet.to(device)
efficientnet = efficientnet.to(device)


In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self, resnet, efficientnet):
        super(EnsembleModel, self).__init__()
        self.resnet = resnet
        self.efficientnet = efficientnet
        self.fc = nn.Linear(7 * 2, 7)  # Combine outputs

    def forward(self, x):
        resnet_out = self.resnet(x)
        effnet_out = self.efficientnet(x)
        combined = torch.cat((resnet_out, effnet_out), dim=1)
        return self.fc(combined)

# Create the ensemble model
ensemble_model = EnsembleModel(resnet, efficientnet).to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble_model.parameters(), lr=0.0001)

num_epochs = 10

for epoch in range(num_epochs):
    ensemble_model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = ensemble_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ Ensemble Training Complete!")


In [ ]:
ensemble_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = ensemble_model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds)
print(f"✅ Ensemble Model Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import torchvision.models as models
from torch.utils.data import DataLoader, random_split

# Load ResNet-50 with pretrained weights
model = models.resnet50(pretrained=True)

# Modify the final layer for 7-class classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 5
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Splitting the dataset into training (80%) and validation (20%)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_train_loss / len(train_loader)
    train_acc = correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_loader)
    val_acc = correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] -> Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")

print("✅ Training Complete!")

# Test Evaluation
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        correct_test += (preds == labels).sum().item()
        total_test += labels.size(0)

test_accuracy = correct_test / total_test
print(f"✅ Final Test Accuracy: {test_accuracy * 100:.2f}%")

# Print final results
print("\n📊 Final Accuracy Scores:")
print(f"🔹 Train Accuracy: {train_accuracies[-1] * 100:.2f}%")
print(f"🔹 Validation Accuracy: {val_accuracies[-1] * 100:.2f}%")
print(f"🔹 Test Accuracy: {test_accuracy * 100:.2f}%")
